## Read data from database

In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [4]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [41]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# table_suffix = ["0","3","9","A","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where (CNT_CODE = 1 or MEM_EXECUTE = 1) and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201704')
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 5.231847524642944 seconds ---
--- 4.660666227340698 seconds ---
--- 3.9280669689178467 seconds ---
--- 2.8787410259246826 seconds ---
--- 3.6401829719543457 seconds ---
--- 3.8073761463165283 seconds ---
--- 3.7070224285125732 seconds ---
--- 3.7544069290161133 seconds ---
--- 3.7250242233276367 seconds ---
--- 3.0903587341308594 seconds ---
--- 3.022162675857544 seconds ---
--- 3.583305835723877 seconds ---
--- 3.7613255977630615 seconds ---
--- 3.684119462966919 seconds ---
--- 3.906905174255371 seconds ---
--- 3.4077231884002686 seconds ---
162542


In [42]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# table_suffix = ["0","3","9","A","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where (CNT_CODE = 1 or MEM_EXECUTE = 1) and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201705')
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

--- 3.953291416168213 seconds ---
--- 4.301469802856445 seconds ---
--- 4.776055097579956 seconds ---
--- 4.221483469009399 seconds ---
--- 4.561415433883667 seconds ---
--- 4.487716913223267 seconds ---
--- 4.43102765083313 seconds ---
--- 4.561986207962036 seconds ---
--- 4.231433868408203 seconds ---
--- 4.492696046829224 seconds ---
--- 4.55763053894043 seconds ---
--- 4.370911359786987 seconds ---
--- 3.9771170616149902 seconds ---
--- 4.1825315952301025 seconds ---
--- 4.036640882492065 seconds ---
--- 4.036452054977417 seconds ---
187577


## Check and split data

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 10000

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines >= 4 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines >= 4 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

In [8]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [9]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

## Autoencoder

In [10]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [63]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, batch_size=32, dim=10000, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, Y = self.__data_generation(list_IDs_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > self.dim:
                bytes_data = [int(single_byte) for single_byte in in_file.read(self.dim)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data

#         X = X.reshape((-1, 100, 100, 1)) / 255.0
        T = X.reshape((-1, 10000)) / 255.0
        Y = X.reshape((-1, 10000, 1)) / 255.0
        return T, Y

In [66]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Conv2D, Multiply, GlobalMaxPooling1D, Dropout, Activation
from keras.layers import UpSampling2D, Flatten, merge, MaxPooling2D, MaxPooling1D, UpSampling1D
from keras.models import load_model, Model
from keras.layers import merge, Dropout, BatchNormalization, Maximum, Add
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split

class Autoencoder():
    def __init__(self, autoencoder_name, encoder_name):
        self.autoencoder_name = autoencoder_name
        self.encoder_name = encoder_name
        self.autoencoder = None
        self.encoder = None
        self.start_time = time.time()

    def get_model(self):
#         input_img = Input(shape=(100, 100, 1))
# Deep
        input_sequence = Input(shape=(10000,));
    
        x = Embedding(256, 8, input_length=10000)(input_sequence)
        x = Activation('relu')(x)
        x = Conv1D(32, 5, padding='same')(x)
        x = Conv1D(32, 5, padding='causal', dilation_rate=1)(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=2)(x)
        x = MaxPooling1D(2, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=4)(x)
        x = MaxPooling1D(2, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=8)(x)
        x = MaxPooling1D(2, padding='same')(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=16)(x)
        x = MaxPooling1D(5, padding='same')(x)
        encoded = Dense(2, activation='sigmoid')(x)
        
        # at this point the representation is (4, 4, 8) i.e. 128-dimensional
        x = Activation('relu')(encoded)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=16)(x)
        x = UpSampling1D(5)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=8)(x)
        x = UpSampling1D(2)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=4)(x)
        x = UpSampling1D(2)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 3, padding='same')(x)
        x = Conv1D(32, 3, padding='causal', dilation_rate=2)(x)
        x = UpSampling1D(2)(x)
        x = Activation('relu')(x)
        x = Conv1D(32, 5, padding='same')(x)
        x = Conv1D(32, 5, padding='causal', dilation_rate=1)(x)
        x = UpSampling1D(5)(x)
        decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
        
## Deep
#         x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
#         x = MaxPooling2D((2, 2), padding='same')(x)
#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#         x = MaxPooling2D((2, 2), padding='same')(x)
#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#         x = MaxPooling2D((5, 5), padding='same')(x)
#         encoded = Dense(2, activation='sigmoid')(x)
        
#         # at this point the representation is (4, 4, 8) i.e. 128-dimensional

#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
#         x = UpSampling2D((5, 5))(x)
#         x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#         x = UpSampling2D((2, 2))(x)
#         x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#         x = UpSampling2D((2, 2))(x)
#         decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

## Base
#         x = Conv2D(32, 4, activation='relu', padding='same')(input_img)
#         x = MaxPooling2D(25, padding='same')(x)
#         encoded = Dense(1, activation='sigmoid')(x)

#         x = Conv2D(32, 4, activation='relu', padding='same')(encoded)
#         x = UpSampling2D(25)(x)
#         decoded = Conv2D(1, 4, activation='sigmoid', padding='same')(x)

        self.autoencoder = Model(inputs=input_sequence, outputs=decoded)
        self.encoder = Model(inputs=input_sequence, outputs=encoded)

        self.autoencoder.compile(loss='mean_squared_error', optimizer=RMSprop())
#         self.autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
        self.autoencoder.summary()

    def train(self, train_df, max_epoch, batch_size=32):
        self.get_model()
        partition_train, partition_validation = train_test_split(range(len(train_df)), test_size=0.05,
                                                                 random_state=1234)
        print('Length of the train: ', len(partition_train))
        print('Length of the validation: ', len(partition_validation))

        #         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/autoencoder/models/"+ str(self.start_time) +"-{epoch:04d}-{val_loss:.5f}_12_3_a_0.h5"
        #         early_stopping = EarlyStopping("val_loss", patience=2, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point]
        
        # Generators
        training_generator = DataGenerator(partition_train, train_df, batch_size)
        validation_generator = DataGenerator(partition_validation, train_df, batch_size)

        self.autoencoder.fit_generator(generator=training_generator,
                                       validation_data=validation_generator,
                                       use_multiprocessing=True,
                                       epochs=max_epoch,
                                       workers=6,
                                       callbacks=callbacks_list)
        self.autoencoder.save(self.autoencoder_name)
        self.encoder.save(self.encoder_name)
        
        
autoencoder = Autoencoder('autoencoder.h5', 'encoder.h5')
autoencoder.train(x_train, max_epoch=64, batch_size=16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 10000)             0         
_________________________________________________________________
embedding_23 (Embedding)     (None, 10000, 8)          2048      
_________________________________________________________________
activation_241 (Activation)  (None, 10000, 8)          0         
_________________________________________________________________
conv1d_509 (Conv1D)          (None, 10000, 32)         1312      
_________________________________________________________________
conv1d_510 (Conv1D)          (None, 10000, 32)         5152      
_________________________________________________________________
max_pooling1d_123 (MaxPoolin (None, 2000, 32)          0         
_________________________________________________________________
activation_242 (Activation)  (None, 2000, 32)          0         
__________

8685/8685 [==============================] - 882s 102ms/step - loss: 0.0756 - val_loss: 0.0762

Epoch 00010: val_loss did not improve from 0.07620
Epoch 11/64
8685/8685 [==============================] - 877s 101ms/step - loss: 0.0755 - val_loss: 0.0758

Epoch 00011: val_loss improved from 0.07620 to 0.07584, saving model to /home/zhaoqi/autoencoder/models/1533489116.5710807-0011-0.07584_12_3_a_0.h5
Epoch 12/64
8685/8685 [==============================] - 880s 101ms/step - loss: 0.0754 - val_loss: 0.0761

Epoch 00012: val_loss did not improve from 0.07584
Epoch 13/64
8685/8685 [==============================] - 876s 101ms/step - loss: 0.0753 - val_loss: 0.0765

Epoch 00013: val_loss did not improve from 0.07584
Epoch 14/64
8685/8685 [==============================] - 865s 100ms/step - loss: 0.0752 - val_loss: 0.0760

Epoch 00014: val_loss did not improve from 0.07584
Epoch 15/64
8685/8685 [==============================] - 866s 100ms/step - loss: 0.0751 - val_loss: 0.0759

Epoch 00015:

Process ForkPoolWorker-879:
Process ForkPoolWorker-882:
Process ForkPoolWorker-880:
Process ForkPoolWorker-878:
Process ForkPoolWorker-883:
Process ForkPoolWorker-881:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5

KeyboardInterrupt: 

In [ ]:
from keras.models import Model

model_dir = '/home/zhaoqi/autoencoder/models/'
f_name = '1533522697.911087-0057-0.06419_12_3_a_0.h5'
c_model = load_model(model_dir + f_name)

model_f = Model(c_model.input, c_model.layers[-21].output)

model_f.summary()

train_generator = DataGenerator(range(len(x_train)), x_train, 16, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, 16, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, 16, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

In [75]:
malcon_train_x = malcon_train_x.reshape(len(malcon_train_x), -1)
malcon_val_x = malcon_val_x.reshape(len(malcon_val_x), -1)
malcon_test_x = malcon_test_x.reshape(len(malcon_test_x), -1)

In [76]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [77]:
def get_max_feature(origin_data, feature_data):
    feature_data = pd.DataFrame(feature_data)
    feature_data['mw_file_hash'] = origin_data.iloc[0:len(feature_data)][['mw_file_hash']]
    feature_data = feature_data.groupby('mw_file_hash').max().merge(origin_data[['mw_file_hash','mw_em_f','mw_num_engines']], how='inner', on='mw_file_hash')
    tmp_ember = get_ember_feature(feature_data[['mw_em_f']])
    tmp_label = feature_data[['mw_num_engines']].mw_num_engines
    tmp_data = feature_data.drop(['mw_num_engines', 'mw_em_f', 'mw_file_hash'], axis=1)
    return tmp_data, tmp_label, tmp_ember

In [78]:
max_train_data, max_train_label, max_train_ember = get_max_feature(x_train, malcon_train_x)
max_val_data, max_val_label, max_val_ember = get_max_feature(x_val, malcon_val_x)
max_test_data, max_test_label, max_test_ember = get_max_feature(x_test, malcon_test_x)

KeyboardInterrupt: 

In [80]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 50+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:50] = m_data.iloc[index]
        m_x[index, 50:50+2351] = e_data[index]  
    return m_x

In [81]:
merge_train_x = merge_feature(max_train_data, max_train_ember)
merge_val_x = merge_feature(max_val_data, max_val_ember)
merge_test_x = merge_feature(max_test_data, max_test_ember)

In [86]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, valid_sets=valid_sets, num_boost_round=10000, early_stopping_rounds=10)
    y_pred = model.predict(x_val, num_iteration=model.best_iteration)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [87]:
model_m = get_model(merge_train_x[:,:], max_train_label, merge_val_x[:,:], max_val_label )

[1]	valid_0's binary_logloss: 0.581733
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.518383
[3]	valid_0's binary_logloss: 0.467749
[4]	valid_0's binary_logloss: 0.422848
[5]	valid_0's binary_logloss: 0.385547
[6]	valid_0's binary_logloss: 0.352819
[7]	valid_0's binary_logloss: 0.323956
[8]	valid_0's binary_logloss: 0.298528
[9]	valid_0's binary_logloss: 0.276523
[10]	valid_0's binary_logloss: 0.256983
[11]	valid_0's binary_logloss: 0.238856
[12]	valid_0's binary_logloss: 0.222173
[13]	valid_0's binary_logloss: 0.207248
[14]	valid_0's binary_logloss: 0.194787
[15]	valid_0's binary_logloss: 0.18286
[16]	valid_0's binary_logloss: 0.173031
[17]	valid_0's binary_logloss: 0.162902
[18]	valid_0's binary_logloss: 0.154537
[19]	valid_0's binary_logloss: 0.146415
[20]	valid_0's binary_logloss: 0.139501
[21]	valid_0's binary_logloss: 0.13338
[22]	valid_0's binary_logloss: 0.127167
[23]	valid_0's binary_logloss: 0.121614
[24]	valid_0's binary_logloss

[200]	valid_0's binary_logloss: 0.0223153
[201]	valid_0's binary_logloss: 0.0222235
[202]	valid_0's binary_logloss: 0.0222071
[203]	valid_0's binary_logloss: 0.0222045
[204]	valid_0's binary_logloss: 0.0222346
[205]	valid_0's binary_logloss: 0.0221861
[206]	valid_0's binary_logloss: 0.0222075
[207]	valid_0's binary_logloss: 0.0221068
[208]	valid_0's binary_logloss: 0.0221132
[209]	valid_0's binary_logloss: 0.0220833
[210]	valid_0's binary_logloss: 0.0220365
[211]	valid_0's binary_logloss: 0.0219935
[212]	valid_0's binary_logloss: 0.0219894
[213]	valid_0's binary_logloss: 0.0220137
[214]	valid_0's binary_logloss: 0.0220177
[215]	valid_0's binary_logloss: 0.0219722
[216]	valid_0's binary_logloss: 0.0220012
[217]	valid_0's binary_logloss: 0.0219874
[218]	valid_0's binary_logloss: 0.0218965
[219]	valid_0's binary_logloss: 0.0218642
[220]	valid_0's binary_logloss: 0.0219216
[221]	valid_0's binary_logloss: 0.0219158
[222]	valid_0's binary_logloss: 0.0218828
[223]	valid_0's binary_logloss: 0.

In [88]:
y_p = model_m.predict(merge_test_x[:,:])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

loss : 0.06704
auc score : 0.99724
accuracy score : 0.97945
thre: 0.9968107390
fp:  0.0009359047
recall:  0.8009765855


(0.997240157779064, 0.06703657879012284, 0.8009765854806448)

In [70]:
model_m = get_model(merge_train_x[:,100:], max_train_label, merge_val_x[:,100:], max_val_label )

NameError: name 'get_model' is not defined

In [69]:
y_p = model_m.predict(merge_test_x[:,100:])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

NameError: name 'model_m' is not defined